This notebook works as deployment for my LSTM model to forecast next hour's load for the Connecticut ISO Zone. 

It makes use of the Voila framework to create a webpage where a user can upload a CSV with the past 9 or more hour's average load and temperature, along with a variable for if the next hour is an on or off peak time dummy, and returns the forecast for the next hour's average energy demand. 

To run: should the user have cloned my repository and have all the required packages installed in their VE, simply cd into where the repository is located on one's system and enter 'voila deployment.ipynb'. This will bring up the aforementioned webpage. 

You can use the file labeled 'dataDeployment.csv' in the /data folder to test. This csv holds values for the three variables for hours 13-22 of 10/19/21.

In [1]:
# Libraries
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import io
from ipywidgets import FileUpload, Label, Image, Button, Output
import os

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import load_model

In [2]:
from platform import python_version

print(python_version())

3.8.12


Begin by loading in the trained model.

In [3]:
lstm_model = load_model('loadFcastLSTM.h5')

2022-01-18 12:40:28.768995: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Upload CSV
csv_upload = FileUpload(accept='*.csv', multiple=False)
csv_label = Label(value = "Please upload a single csv below.")

display(csv_label, csv_upload)

Label(value='Please upload a single csv below.')

FileUpload(value={}, accept='*.csv', description='Upload')

In [5]:
input_file = list(csv_upload.value.values())[0]
content = input_file['content']
content = io.StringIO(content.decode('utf-8'))
df = pd.read_csv(content)

predict_button = Button(description="Predict!")
result = Output()

In [6]:
LoadMW_Plus1 = df["LoadMW"].shift(periods=-1)
LoadMW_Plus1.name = 'LoadMW_Plus1'
df = pd.concat([df, LoadMW_Plus1], axis=1)

In [7]:
null_index = df['LoadMW_Plus1'].isnull()

In [8]:
def predict(button_info):
    try:
        df_scaled = df.copy()

        # Initialize scaler (here we use Min/Max)
        sc = MinMaxScaler()

        # Fit
        sc_x = StandardScaler().fit(df_scaled[["TempF","LoadMW"]])
        sc_y = StandardScaler().fit(df_scaled.loc[~null_index, ['LoadMW_Plus1']].values.reshape(-1,1))

        # Scale numerical variables
        df_scaled[["TempF","LoadMW"]] = sc_x.transform(df_scaled[["TempF","LoadMW"]])
        df_scaled.loc[~null_index, ["LoadMW_Plus1"]] = sc_y.transform(df_scaled.loc[~null_index, ['LoadMW_Plus1']].values.reshape(-1,1))

        # Remove time objects from scaled dataframe
        df_scaled = df_scaled[["Peak_Plus1","TempF","LoadMW","LoadMW_Plus1"]]

        # Reshape dataframe 
        # Put our timeseries variables into arrays and horizontally stack them
        # This input format is necessary to be able to run through the split_sequences() function
        in_seq1 = np.array(df_scaled["LoadMW"])
        in_seq2 = np.array(df_scaled["TempF"])
        in_seq3 = np.array(df_scaled["Peak_Plus1"])
        out_seq = np.array(df_scaled["LoadMW_Plus1"])

        # convert to [rows, columns] structure
        in_seq1 = in_seq1.reshape((len(in_seq1), 1))
        in_seq2 = in_seq2.reshape((len(in_seq2), 1))
        in_seq3 = in_seq3.reshape((len(in_seq3), 1)) 
        out_seq = out_seq.reshape((len(out_seq), 1))

        # horizontally stack columns
        dataset = np.hstack((in_seq1, in_seq2, in_seq3, out_seq))

        # The function below splits our dataset into samples of sequences of time stpes
        # It is taken from machinelearningmastery
        def split_sequences(sequences, n_steps):
          X, y = list(), list()
          for i in range(len(sequences)):
            # find the end of this pattern
            end_ix = i + n_steps
            # check if we are beyond the dataset
            if end_ix > len(sequences):
              break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
            X.append(seq_x)
            y.append(seq_y)
          return np.array(X), np.array(y)


        X, y = split_sequences(dataset, 9)

        prediction = sc_y.inverse_transform(lstm_model.predict(X[1].reshape(1,9,3))).item()

        with result:
            print(f"The load forecast for the next hour is {prediction}.")
    except Exception as e:
        with result:
            print(e)

In [9]:
predict_button.on_click(predict)
display(predict_button, result)

Button(description='Predict!', style=ButtonStyle())

Output()

2022-01-18 12:40:38.918476: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
